In [2]:
"""
The objective is to identify Pokemon which are optimally suited to defeating the Elite Four and Champion
in Pokemon FireRed Version, hereafter referred to as "E4".

These are the high-level steps to achieving this goal:
1. Get type combinations of each E4 Pokemon.
2. Get distinct type combinations of all Pokemon in FireRed.
3. Determine the types over which each other type has offensive and/or defensive advantage.
4. Score type combinations on how advantageous they are against E4 type combinations.
5. Find Pokemon with the highest-rated types and rank them by their total stats.
6. Find the party lineup with the best overall advantage against E4.
"""

'\nThe objective is to identify Pokemon which are optimally suited to defeating the Elite Four and Champion, hereafter referred to as "E4".\n\nThese are the high-level steps to achieving this goal:\n1. Get type combinations of each E4 Pokemon.\n2. Get distinct type combinations of all Pokemon in Fire Red.\n3. Determine the types over which each other type has offensive and/or defensive advantage.\n4. Score type combinations on how advantageous they are against E4 type combinations.\n5. Find Pokemon with the highest-rated types and rank them by their stats.\n\nWe will need the Pokedex data, the Type chart, and the E4 party lists.\n\nSuitability scores will be 0-8.\n"attacker" = "the trainer\'s Pokemon"\n"defender" = "the E4 Pokemon"\n"at1" = "attacker type 1"\n"at2" = "attacker type 2"\netc.\n\nIf at1 has off. advantage over dt1 then +1 else 0.\nIf at1 has off. advantage over dt2 then +1 else 0.\nIf at1 has def. advantage over dt1 then +1 else 0.\nIf at1 has def. advantage over dt2 then

In [3]:
# Import Modules
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings

# Disable all warnings
warnings.filterwarnings('ignore')

In [4]:
# Get Pokedex Data
page_to_scrape = requests.get("https://pokemondb.net/pokedex/all")
soup = BeautifulSoup(page_to_scrape.content, "html.parser")
pokedex = soup.find('table', attrs={'id': 'pokedex'})

# Convert scraped table to Pandas df
pokedex_df = pd.read_html(str(pokedex))[0].set_index('#')

# Filter to Kanto Region and Drop Mega Evolutions
pokedex_df = pokedex_df[pokedex_df.index < 150]
pokedex_df = pokedex_df[~pokedex_df.index.duplicated(keep='first')]

# Split Type column
pokedex_df[['Type 1', 'Type 2']] = pokedex_df['Type'].str.split(n=1, expand=True)
pokedex_df.drop(columns='Type', inplace=True)

# Replace None in Type 2 with Type 1
pokedex_df.loc[pokedex_df['Type 2'].isna(), 'Type 2'] = pokedex_df.loc[pokedex_df['Type 2'].isna(), 'Type 1']

# Correct entries of Fairy type (not present in FireRed)
pokedex_df.loc[35, 'Type 1'] = 'Normal'
pokedex_df.loc[35, 'Type 2'] = 'Normal'
pokedex_df.loc[36, 'Type 1'] = 'Normal'
pokedex_df.loc[36, 'Type 2'] = 'Normal'
pokedex_df.loc[39, 'Type 1'] = 'Normal'
pokedex_df.loc[39, 'Type 2'] = 'Normal'
pokedex_df.loc[40, 'Type 1'] = 'Normal'
pokedex_df.loc[40, 'Type 2'] = 'Normal'
pokedex_df.loc[122, 'Type 1'] = 'Psychic'
pokedex_df.loc[122, 'Type 2'] = 'Psychic'

display(pokedex_df.head(10))

,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
#,,,,,,,,,,
1,Bulbasaur,318,45,49,49,65,65,45,Grass,Poison
2,Ivysaur,405,60,62,63,80,80,60,Grass,Poison
3,Venusaur,525,80,82,83,100,100,80,Grass,Poison
4,Charmander,309,39,52,43,60,50,65,Fire,Fire
5,Charmeleon,405,58,64,58,80,65,80,Fire,Fire
6,Charizard,534,78,84,78,109,85,100,Fire,Flying
7,Squirtle,314,44,48,65,50,64,43,Water,Water
8,Wartortle,405,59,63,80,65,80,58,Water,Water
9,Blastoise,530,79,83,100,85,105,78,Water,Water


In [5]:
# Get Type chart
page_to_scrape = requests.get("https://pokemondb.net/type")
soup = BeautifulSoup(page_to_scrape.content, "html.parser")
type = soup.find('table', attrs={'class': 'type-table'})

# Convert scraped table to Pandas df
type_df = pd.read_html(str(type))[0]

# Drop Fairy type (not present in Gen 3)
del type_df['Fai']
type_df = type_df.drop(index=17)

# Rewrite 1/2 and fill empty
type_df = type_df.map(lambda x: x.replace('½', '0.5') if isinstance(x, str) else x)
type_df = type_df.fillna(1)

# Rename columns
type_df.rename(columns={
    'Nor': 'Normal',
    'Fir': 'Fire',
    'Wat': 'Water',
    'Ele': 'Electric',
    'Gra': 'Grass',
    'Fig': 'Fighting',
    'Poi': 'Poison',
    'Gro': 'Ground',
    'Fly': 'Flying',
    'Psy': 'Psychic',
    'Roc': 'Rock',
    'Gho': 'Ghost',
    'Dra': 'Dragon',
    'Dar': 'Dark',
    'Ste': 'Steel'
}, inplace=True)

column_names = type_df.columns.to_list()
column_names[0] = 'Attacker'
type_df.columns = column_names

# Correct Ghost and Dark to do 1/2 damage to Steel types
type_df.loc[13, 'Steel'] = 0.5
type_df.loc[15, 'Steel'] = 0.5

# Set numeric cols to float
type_df[[x for x in type_df.columns[1:]]] = type_df[[x for x in type_df.columns[1:]]].astype(float)

display(type_df)

,Attacker,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel
0,Normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5
1,Fire,1.0,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0
2,Water,1.0,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0
3,Electric,1.0,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0
4,Grass,1.0,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5
5,Ice,1.0,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5
6,Fighting,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0
7,Poison,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0
8,Ground,1.0,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0
9,Flying,1.0,1.0,1.0,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,0.5


In [6]:
# Create dictionaries of attack and defense advantages
atk_adv = {type: [] for type in type_df['Attacker']}
def_adv = {type: [] for type in type_df['Attacker']}

for idx, row in type_df.iloc[:,1:].iterrows():
    for col, val in row.items():
        if val > 1:
            atk_adv[type_df['Attacker'][idx]].append(col)
        elif val < 1:
            def_adv[col].append(type_df['Attacker'][idx])

print(atk_adv)
print(def_adv)

{'Normal': [], 'Fire': ['Grass', 'Ice', 'Bug', 'Steel'], 'Water': ['Fire', 'Ground', 'Rock'], 'Electric': ['Water', 'Flying'], 'Grass': ['Water', 'Ground', 'Rock'], 'Ice': ['Grass', 'Ground', 'Flying', 'Dragon'], 'Fighting': ['Normal', 'Ice', 'Rock', 'Dark', 'Steel'], 'Poison': ['Grass'], 'Ground': ['Fire', 'Electric', 'Poison', 'Rock', 'Steel'], 'Flying': ['Grass', 'Fighting', 'Bug'], 'Psychic': ['Fighting', 'Poison'], 'Bug': ['Grass', 'Psychic', 'Dark'], 'Rock': ['Fire', 'Ice', 'Flying', 'Bug'], 'Ghost': ['Psychic', 'Ghost'], 'Dragon': ['Dragon'], 'Dark': ['Psychic', 'Ghost'], 'Steel': ['Ice', 'Rock']}
{'Normal': ['Ghost'], 'Fire': ['Fire', 'Grass', 'Ice', 'Bug', 'Steel'], 'Water': ['Fire', 'Water', 'Ice', 'Steel'], 'Electric': ['Electric', 'Flying', 'Steel'], 'Grass': ['Water', 'Electric', 'Grass', 'Ground'], 'Ice': ['Ice'], 'Fighting': ['Bug', 'Rock', 'Dark'], 'Poison': ['Grass', 'Fighting', 'Poison', 'Bug'], 'Ground': ['Electric', 'Poison', 'Rock'], 'Flying': ['Grass', 'Fighting',

In [7]:
# Get distinct type combinations from Pokedex data
type_combinations = pokedex_df[['Type 1', 'Type 2']].drop_duplicates()
display(type_combinations)

,Type 1,Type 2
#,,
1,Grass,Poison
4,Fire,Fire
6,Fire,Flying
7,Water,Water
10,Bug,Bug
12,Bug,Flying
13,Bug,Poison
16,Normal,Flying
19,Normal,Normal


In [8]:
def score_types(opponent_df):
    """
    Input: A df containing Pokemon to be beaten
    Output: A df with the best type combinations to match up against the type combinations 
            of the Pokemon to be beaten

    A score is assigned to each matchup, adding a point every time one of the types of the 
    trainer's Pokemon has an offensive or defensive advantage over one of the types of the 
    opponent's Pokemon, and subtracting a point for disadvantage. Scores range from -8 to 8.

    This function also assumes Bulbasaur as a starter.
    """

    # Construct a dictionary to hold the output data
    score_dict = {
        'Opponent Type 1': [],
        'Opponent Type 2': [],
        'Trainer Type 1': [],
        'Trainer Type 2': [],
        'Score': []
    }

    # Match up types and assign scores
    for i, x in opponent_df[['Type 1', 'Type 2']].iterrows():
        for j, y in type_combinations.iterrows():
            score = 0
            score += x[0] in atk_adv[y[0]]
            score -= y[0] in atk_adv[x[0]]
            score += x[0] in atk_adv[y[1]]
            score -= y[1] in atk_adv[x[0]]
            score += x[1] in atk_adv[y[0]]
            score -= y[0] in atk_adv[x[1]]
            score += x[1] in atk_adv[y[1]]
            score -= y[1] in atk_adv[x[1]]
            score += x[0] in def_adv[y[0]]
            score -= y[0] in def_adv[x[0]]
            score += x[0] in def_adv[y[1]]
            score -= y[1] in def_adv[x[0]]
            score += x[1] in def_adv[y[0]]
            score -= y[0] in def_adv[x[1]]
            score += x[1] in def_adv[y[1]]
            score -= y[1] in def_adv[x[1]]
            
            # Add data to dictionary
            score_dict['Opponent Type 1'].append(x[0])
            score_dict['Opponent Type 2'].append(x[1])
            score_dict['Trainer Type 1'].append(y[0])
            score_dict['Trainer Type 2'].append(y[1])
            score_dict['Score'].append(score)

    # Convert dictionary to DataFrame
    df = pd.DataFrame(score_dict)

    # Remove scores less than one
    df = df[df['Score'] > 0]

    # Match Pokemon to the scored types
    df = pd.merge(df, pokedex_df, left_on=['Trainer Type 1', 'Trainer Type 2'], right_on=['Type 1', 'Type 2'], how='left')

    # Filter by stat totals
    max_values = df.groupby(['Type 1', 'Type 2'])['Total'].transform('max')
    df = df[df['Total'] == max_values]

    # Remove Pokemon exlusive to LeafGreen, other starters, and Pokemon available only through trade.
    pokemon_to_remove = [
        'Sandshrew',
        'Sandslash',
        'Vulpix',
        'Ninetales',
        'Bellsprout',
        'Weepinbell',
        'Victreebel',
        'Slowpoke',
        'Slowbro',
        'Staryu',
        'Starmie',
        'Magmar',
        'Pinsir',
        'Alakazam',
        'Machamp',
        'Golem',
        'Gengar',
        'Squirtle',
        'Wartortle',
        'Blastoise',
        'Charmander',
        'Charmeleon',
        'Charizard'    
    ]

    # Filter out Pokemon from above list and order by opponent type, score, and total stats
    df = df[~df['Name'].isin(pokemon_to_remove)].sort_values(by=['Opponent Type 1', 'Opponent Type 2', 'Score', 'Total'], ascending=False)

    # Rank by score, break ties with total stats
    df['Rank'] = df.groupby(['Opponent Type 1', 'Opponent Type 2']).cumcount() + 1

    # Filter to top-ranked and drop rank column
    final_df = df[df['Rank'] == 1].drop(columns=['Rank'])

    return final_df

    



In [9]:
# Construct E4 data, assuming Bulbasaur as player's starter (Champion's lineup will be different depending)
lorelei = {
    'Name': [
        'Dewgong',
        'Cloyster',
        'Slowbro',
        'Jynx',
        'Lapras'
    ]
}

bruno = {
    'Name': [
        'Onix',
        'Hitmonchan',
        'Hitmonlee',
        'Onix',
        'Machamp'
    ]
}

agatha = {
    'Name': [
        'Gengar',
        'Golbat',
        'Haunter',
        'Arbok',
        'Gengar'
    ]
}

lance = {
    'Name': [
        'Gyarados',
        'Dragonair',
        'Dragonair',
        'Aerodactyl',
        'Dragonite'
    ]
}

champ = {
    'Name': [
        'Pidgeot',
        'Alakazam',
        'Rhydon',
        'Exeggutor',
        'Gyarados',
        'Charizard'
    ]
}

# Convert data dicts to dfs
lorelei_df = pd.DataFrame(lorelei).merge(pokedex_df)
bruno_df = pd.DataFrame(bruno).merge(pokedex_df)
agatha_df = pd.DataFrame(agatha).merge(pokedex_df)
lance_df = pd.DataFrame(lance).merge(pokedex_df)
champ_df = pd.DataFrame(champ).merge(pokedex_df)

display(lorelei_df)
display(bruno_df)
display(agatha_df)
display(lance_df)
display(champ_df)

,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,Dewgong,475,90,70,80,70,95,70,Water,Ice
1,Cloyster,525,50,95,180,85,45,70,Water,Ice
2,Slowbro,490,95,75,110,100,80,30,Water,Psychic
3,Jynx,455,65,50,35,115,95,95,Ice,Psychic
4,Lapras,535,130,85,80,85,95,60,Water,Ice


,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,Onix,385,35,45,160,30,45,70,Rock,Ground
1,Hitmonchan,455,50,105,79,35,110,76,Fighting,Fighting
2,Hitmonlee,455,50,120,53,35,110,87,Fighting,Fighting
3,Onix,385,35,45,160,30,45,70,Rock,Ground
4,Machamp,505,90,130,80,65,85,55,Fighting,Fighting


,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,Gengar,500,60,65,60,130,75,110,Ghost,Poison
1,Golbat,455,75,80,70,65,75,90,Poison,Flying
2,Haunter,405,45,50,45,115,55,95,Ghost,Poison
3,Arbok,448,60,95,69,65,79,80,Poison,Poison
4,Gengar,500,60,65,60,130,75,110,Ghost,Poison


,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,Gyarados,540,95,125,79,60,100,81,Water,Flying
1,Dragonair,420,61,84,65,70,70,70,Dragon,Dragon
2,Dragonair,420,61,84,65,70,70,70,Dragon,Dragon
3,Aerodactyl,515,80,105,65,60,75,130,Rock,Flying
4,Dragonite,600,91,134,95,100,100,80,Dragon,Flying


,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,Pidgeot,479,83,80,75,70,70,101,Normal,Flying
1,Alakazam,500,55,50,45,135,95,120,Psychic,Psychic
2,Rhydon,485,105,130,120,45,45,40,Ground,Rock
3,Exeggutor,530,95,95,85,125,75,55,Grass,Psychic
4,Gyarados,540,95,125,79,60,100,81,Water,Flying
5,Charizard,534,78,84,78,109,85,100,Fire,Flying


In [13]:
# Create table showing top-scored Pokemon against each of Lorelei's Pokemon.
lorelei_types = lorelei_df[['Name', 'Type 1', 'Type 2']].merge(score_types(lorelei_df), left_on=['Type 1', 'Type 2'], right_on=['Opponent Type 1', 'Opponent Type 2'], how='left')
display(lorelei_types)

# Show distinct Pokemon selected for use against Lorelei.
print(set(lorelei_types['Name_y']))

,Name_x,Type 1_x,Type 2_x,Opponent Type 1,Opponent Type 2,Trainer Type 1,Trainer Type 2,Score,Name_y,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1_y,Type 2_y
0,Dewgong,Water,Ice,Water,Ice,Electric,Electric,2,Jolteon,525,65,65,60,110,95,130,Electric,Electric
1,Cloyster,Water,Ice,Water,Ice,Electric,Electric,2,Jolteon,525,65,65,60,110,95,130,Electric,Electric
2,Slowbro,Water,Psychic,Water,Psychic,Grass,Grass,4,Tangela,435,65,55,115,100,40,60,Grass,Grass
3,Jynx,Ice,Psychic,Ice,Psychic,Fire,Fire,4,Arcanine,555,90,110,80,100,80,95,Fire,Fire
4,Lapras,Water,Ice,Water,Ice,Electric,Electric,2,Jolteon,525,65,65,60,110,95,130,Electric,Electric


{'Jolteon', 'Arcanine', 'Tangela'}


In [19]:
# Create table showing top-scored Pokemon against each of Bruno's Pokemon.
bruno_types = bruno_df[['Name', 'Type 1', 'Type 2']].merge(score_types(bruno_df), left_on=['Type 1', 'Type 2'], right_on=['Opponent Type 1', 'Opponent Type 2'], how='left')
display(bruno_types)

# Show distinct Pokemon selected for use against Bruno.
print(set(bruno_types['Name_y']))

,Name_x,Type 1_x,Type 2_x,Opponent Type 1,Opponent Type 2,Trainer Type 1,Trainer Type 2,Score,Name_y,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1_y,Type 2_y
0,Onix,Rock,Ground,Rock,Ground,Grass,Grass,6,Tangela,435,65,55,115,100,40,60,Grass,Grass
1,Hitmonchan,Fighting,Fighting,Fighting,Fighting,Poison,Flying,6,Golbat,455,75,80,70,65,75,90,Poison,Flying
2,Hitmonlee,Fighting,Fighting,Fighting,Fighting,Poison,Flying,6,Golbat,455,75,80,70,65,75,90,Poison,Flying
3,Onix,Rock,Ground,Rock,Ground,Grass,Grass,6,Tangela,435,65,55,115,100,40,60,Grass,Grass
4,Machamp,Fighting,Fighting,Fighting,Fighting,Poison,Flying,6,Golbat,455,75,80,70,65,75,90,Poison,Flying


{'Golbat', 'Tangela'}


In [15]:
# Create table showing top-scored Pokemon against each of Agatha's Pokemon.
agatha_types = agatha_df[['Name', 'Type 1', 'Type 2']].merge(score_types(agatha_df), left_on=['Type 1', 'Type 2'], right_on=['Opponent Type 1', 'Opponent Type 2'], how='left')
display(agatha_types)

# Show distinct Pokemon selected for use against Agatha.
print(set(agatha_types['Name_y']))

,Name_x,Type 1_x,Type 2_x,Opponent Type 1,Opponent Type 2,Trainer Type 1,Trainer Type 2,Score,Name_y,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1_y,Type 2_y
0,Gengar,Ghost,Poison,Ghost,Poison,Ground,Rock,3,Rhydon,485,105,130,120,45,45,40,Ground,Rock
1,Golbat,Poison,Flying,Poison,Flying,Electric,Electric,4,Jolteon,525,65,65,60,110,95,130,Electric,Electric
2,Haunter,Ghost,Poison,Ghost,Poison,Ground,Rock,3,Rhydon,485,105,130,120,45,45,40,Ground,Rock
3,Arbok,Poison,Poison,Poison,Poison,Ground,Rock,6,Rhydon,485,105,130,120,45,45,40,Ground,Rock
4,Gengar,Ghost,Poison,Ghost,Poison,Ground,Rock,3,Rhydon,485,105,130,120,45,45,40,Ground,Rock


{'Jolteon', 'Rhydon'}


In [16]:
# Create table showing top-scored Pokemon against each of Lance's Pokemon.
lance_types = lance_df[['Name', 'Type 1', 'Type 2']].merge(score_types(lance_df), left_on=['Type 1', 'Type 2'], right_on=['Opponent Type 1', 'Opponent Type 2'], how='left')
display(lance_types)

# Show distinct Pokemon selected for use against Lance.
print(set(lance_types['Name_y']))

,Name_x,Type 1_x,Type 2_x,Opponent Type 1,Opponent Type 2,Trainer Type 1,Trainer Type 2,Score,Name_y,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1_y,Type 2_y
0,Gyarados,Water,Flying,Water,Flying,Electric,Electric,6,Jolteon,525,65,65,60,110,95,130,Electric,Electric
1,Dragonair,Dragon,Dragon,Dragon,Dragon,Ice,Flying,2,Articuno,580,90,85,100,95,125,85,Ice,Flying
2,Dragonair,Dragon,Dragon,Dragon,Dragon,Ice,Flying,2,Articuno,580,90,85,100,95,125,85,Ice,Flying
3,Aerodactyl,Rock,Flying,Rock,Flying,Electric,Steel,5,Magneton,465,50,60,95,120,70,70,Electric,Steel
4,Dragonite,Dragon,Flying,Dragon,Flying,Electric,Steel,3,Magneton,465,50,60,95,120,70,70,Electric,Steel


{'Magneton', 'Jolteon', 'Articuno'}


In [17]:
# Create table showing top-scored Pokemon against each of The Champion's Pokemon.
champ_types = champ_df[['Name', 'Type 1', 'Type 2']].merge(score_types(champ_df), left_on=['Type 1', 'Type 2'], right_on=['Opponent Type 1', 'Opponent Type 2'], how='left')
display(champ_types)

# Show distinct Pokemon selected for use against The Champion.
print(set(champ_types['Name_y']))

,Name_x,Type 1_x,Type 2_x,Opponent Type 1,Opponent Type 2,Trainer Type 1,Trainer Type 2,Score,Name_y,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1_y,Type 2_y
0,Pidgeot,Normal,Flying,Normal,Flying,Electric,Electric,4,Jolteon,525,65,65,60,110,95,130,Electric,Electric
1,Alakazam,Psychic,Psychic,Psychic,Psychic,Bug,Flying,2,Scyther,500,70,110,80,55,80,105,Bug,Flying
2,Rhydon,Ground,Rock,Ground,Rock,Grass,Grass,6,Tangela,435,65,55,115,100,40,60,Grass,Grass
3,Exeggutor,Grass,Psychic,Grass,Psychic,Bug,Flying,5,Scyther,500,70,110,80,55,80,105,Bug,Flying
4,Gyarados,Water,Flying,Water,Flying,Electric,Electric,6,Jolteon,525,65,65,60,110,95,130,Electric,Electric
5,Charizard,Fire,Flying,Fire,Flying,Rock,Water,6,Omastar,495,70,60,125,115,70,55,Rock,Water


{'Scyther', 'Jolteon', 'Tangela', 'Omastar'}


In [22]:
# Construct total E4 data
e4 = {
    'Name': [
        'Dewgong',
        'Cloyster',
        'Slowbro',
        'Jynx',
        'Lapras',
        'Onix',
        'Hitmonchan',
        'Hitmonlee',
        'Onix',
        'Machamp',
        'Gengar',
        'Golbat',
        'Haunter',
        'Arbok',
        'Gengar',
        'Gyarados',
        'Dragonair',
        'Dragonair',
        'Aerodactyl',
        'Dragonite',
        'Pidgeot',
        'Alakazam',
        'Rhydon',
        'Exeggutor',
        'Gyarados',
        'Charizard'
    ]
}

# Convert data dicts to dfs
e4_df = pd.DataFrame(e4).merge(pokedex_df)
e4_party = score_types(e4_df)
e4_types = e4_df[['Name', 'Type 1', 'Type 2']].merge(score_types(e4_df), left_on=['Type 1', 'Type 2'], right_on=['Opponent Type 1', 'Opponent Type 2'], how='left')
display(e4_types)
display(e4_types['Name_y'].value_counts())
print(set(e4_types['Name_y']))

,Name_x,Type 1_x,Type 2_x,Opponent Type 1,Opponent Type 2,Trainer Type 1,Trainer Type 2,Score,Name_y,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1_y,Type 2_y
0,Dewgong,Water,Ice,Water,Ice,Electric,Electric,2,Jolteon,525,65,65,60,110,95,130,Electric,Electric
1,Cloyster,Water,Ice,Water,Ice,Electric,Electric,2,Jolteon,525,65,65,60,110,95,130,Electric,Electric
2,Slowbro,Water,Psychic,Water,Psychic,Grass,Grass,4,Tangela,435,65,55,115,100,40,60,Grass,Grass
3,Jynx,Ice,Psychic,Ice,Psychic,Fire,Fire,4,Arcanine,555,90,110,80,100,80,95,Fire,Fire
4,Lapras,Water,Ice,Water,Ice,Electric,Electric,2,Jolteon,525,65,65,60,110,95,130,Electric,Electric
5,Onix,Rock,Ground,Rock,Ground,Grass,Grass,6,Tangela,435,65,55,115,100,40,60,Grass,Grass
6,Hitmonchan,Fighting,Fighting,Fighting,Fighting,Poison,Flying,6,Golbat,455,75,80,70,65,75,90,Poison,Flying
7,Hitmonlee,Fighting,Fighting,Fighting,Fighting,Poison,Flying,6,Golbat,455,75,80,70,65,75,90,Poison,Flying
8,Onix,Rock,Ground,Rock,Ground,Grass,Grass,6,Tangela,435,65,55,115,100,40,60,Grass,Grass
9,Machamp,Fighting,Fighting,Fighting,Fighting,Poison,Flying,6,Golbat,455,75,80,70,65,75,90,Poison,Flying


Name_y
Jolteon     7
Tangela     4
Rhydon      4
Golbat      3
Articuno    2
Magneton    2
Scyther     2
Arcanine    1
Omastar     1
Name: count, dtype: int64

{'Jolteon', 'Articuno', 'Magneton', 'Scyther', 'Golbat', 'Tangela', 'Rhydon', 'Arcanine', 'Omastar'}


In [ ]:
"""
The findings show that based on the total number of E4 Pokemon for which a given Pokemon is considered the best
matchup, the trainers party should consist of:
1. Jolteon (against all but Bruno)
2. Tangela (against Lorelei, Bruno, and the Champion)
3. Rhydon (against Agatha)
4. Golbat (against Bruno)
Some combination of the following for spots 5 & 6:
Articuno (against Lance)
Magneton (against Lance)
Scyther (against the Champion)
"""